# Geometrie mit Topf
Hier wird eine Geometrie mit mehreren Windungen und Topf generiert.
Der untere Rand ist die Rotationsachse.

## Includes

In [1]:
import sys, os
# Add the ImportExport directory to sys.path
import_export_dir = os.path.join(os.path.join(os.getcwd(), '..'), 'ImportExport')
if import_export_dir not in sys.path:
    sys.path.append(import_export_dir)
from global_parameters import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import numpy as np

## Definition Masse und Parameter
Wir setzen $r = 0.001\,\text{m}$ und $i_c = 10\,\text{A}$.

In [2]:
ic = 10                 # Strom in den Windungen (A)

# Mesh Generation
l_air = 0.1         # Grosse der dargestellten Luft um denn Versuchsaufbau
r_wire = 0.001      # Drahtdurchmesser in Metern
nx_wire = 10        # Anzahl Kabel in X-Richtung
ny_wire = 3         # Anzahl Kabel in Y-Richtung
dxdy_wire = 0.5*r_wire

rA_core = 0.01      # Core Höhe
rI_core = 0.008
l_core = 0.08       # Core Länge
epsZero = 1e-8

pot_offset_x = -0.03
pot_offset_y = 0

pot_width = 0.04
pot_height = 0.04
pot_wall = 0.002
pot_hole = 0.0001
pot_nx_wire = 1        # Anzahl Kabel in X-Richtung
pot_ny_wire = 15         # Anzahl Kabel in Y-Richtung
pot_coil_x = 0.0125        #position coil
pot_coil_y = 0.003        #position coil
pot_waterlevel = 90         # in prozent
waterlevel = pot_height-(pot_height-pot_wall) * (pot_waterlevel/100)


points = [  (0+pot_offset_x,               pot_width+pot_offset_y), 
            (0+pot_offset_x,               pot_width- pot_wall+pot_offset_y),
            (pot_height-pot_wall+pot_offset_x,  pot_width- pot_wall+pot_offset_y),
            (pot_height- pot_wall+pot_offset_x, pot_hole+pot_offset_y),
            (pot_height+pot_offset_x,           pot_hole+pot_offset_y),
            (pot_height+pot_offset_x,           pot_width+pot_offset_y) ]


## Generieren des Meshs

In [3]:
geo = SplineGeometry()
geo.AddRectangle(p1=(-l_air/2,epsZero),
                p2=(l_air/2,l_air/2),
                bcs=["rotsym","outer","outer","outer"],
                leftdomain=1,
                rightdomain=0)

pts = []
for i in range(pot_nx_wire):
    for j in range(pot_ny_wire):
        pts.append([
            i*(2*r_wire+dxdy_wire)+pot_coil_x, 
            dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire) + pot_coil_y
        ])
        geo.AddCircle(
            c=(i*(2*r_wire+dxdy_wire)+pot_coil_x, 
            dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)+ pot_coil_y),
            r=r_wire, bc="inner",
            leftdomain=2,
            rightdomain=1)
pts = np.array(pts)

geo.AddRectangle(p1=(pot_height-pot_wall+pot_offset_x-0.00001,  pot_width- pot_wall+pot_offset_y-0.00001),
                p2=(waterlevel+pot_offset_x,pot_hole+pot_offset_y),
                bc="inner",
                leftdomain=4,
                rightdomain=1)



pnums = []
for p in points:
        pnums.append(geo.AppendPoint(*p))
    
for i in range(len(pnums)):
    geo.Append(["line", pnums[i], pnums[(i+1)%len(pnums)]], leftdomain=3, rightdomain=1, bc="inner")


geo.SetMaterial (1, "air")
geo.SetMaterial (2, "copper")
geo.SetMaterial (3, "core")
geo.SetMaterial (4, "water")
geo.SetDomainMaxH(3,(rA_core-rI_core)/2)

mesh = Mesh(geo.GenerateMesh(maxh=0.0025))
#print(mesh.GetBoundaries())

## Darstellung Mesh mit Materialien
```{table} Topf-Geometrie Materialien
:name: topfMeshMaterials
| Material                             | Farbe             |
|------------------------------|-------------|
| Luft              | blau |
| Kupfer | türkis |
| Topf | gelb |
| Wasser | rot |
```

In [4]:
# Visualisierung
mat_mapping = {name: i+1 for i, name in enumerate(mesh.GetMaterials())}
print(mesh.GetMaterials())
cf = mesh.MaterialCF(mat_mapping)
Draw(cf, mesh, "material_colors")

('air', 'copper', 'core', 'water')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Stromdichte speichern
Die Stromdichte wird gemäss Gleichung {eq}`eq:Jrotsym` berechnet und gespeichert. 

In [5]:
Jimp = CoefficientFunction([0,ic/(r_wire**2*pi),0])
Vstore = L2(mesh, order=4)
gf = GridFunction(Vstore)
gf.Set(Jimp)
#Draw(gf, mesh)
# Now save as above
gf.Save("../ImportExport/JimpTopfGeo.gf")

## Mesh speichern

In [6]:
mesh.ngmesh.Save("../ImportExport/myTopfMesh.vol")